In [1]:
import numpy as np
import math
from PIL import Image
import cv2

# 切割后整个图应该是文字网格

In [2]:
def crop_image(image):
    """Crop largest box from and return as pillow image.
    """
    image = image.resize((image.size[0]//2, image.size[1]//2))  # Shrink to make boarder clear
    image = np.asarray(image)
    # Generate edge image
    edge_img = cv2.GaussianBlur(image, (3,3), 0)
    edge_img = cv2.cvtColor(edge_img, cv2.COLOR_BGR2GRAY)
    edge_img = cv2.Canny(edge_img, 50, 150, apertureSize=3)
    # Find contour
    contours, _ = cv2.findContours(edge_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = sorted(contours, key=cv2.contourArea)[-1]
    epsilon = 0.1*cv2.arcLength(contour, True)
    contour = cv2.approxPolyDP(contour, epsilon, True)
    
    def four_point_transform(image, pts):
        def order_points(pts):
            rect = np.zeros((4, 2), dtype = "float32")

            s = pts.sum(axis = 1)
            rect[0] = pts[np.argmin(s)]
            rect[2] = pts[np.argmax(s)]

            diff = np.diff(pts, axis = 1)
            rect[1] = pts[np.argmin(diff)]
            rect[3] = pts[np.argmax(diff)]

            return rect

        rect = order_points(pts)
        (tl, tr, br, bl) = rect

        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))

        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))

        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype = "float32")

        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

        return warped
    
    paper = four_point_transform(image, contour.reshape(4, 2))
    return Image.fromarray(paper)

crop_image(Image.open('box_detect/10.jpg'))

FileNotFoundError: [Errno 2] No such file or directory: 'box_detect/10.jpg'

In [3]:
image = Image.open('box_detect/1.jpg')
image = image.resize((image.size[0]//2, image.size[1]//2))

img = np.asarray(image)

edge_img = cv2.GaussianBlur(img, (3,3), 0)
edge_img = cv2.cvtColor(edge_img, cv2.COLOR_BGR2GRAY)
edge_img = cv2.Canny(edge_img, 50, 150, apertureSize=3)
Image.fromarray(edge_img)

FileNotFoundError: [Errno 2] No such file or directory: 'box_detect/1.jpg'

In [4]:
# Find contours
contours, _ = cv2.findContours(edge_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = sorted(contours, key=cv2.contourArea)[-1]
epsilon = 0.1*cv2.arcLength(contour, True)
contour = cv2.approxPolyDP(contour, epsilon, True)

contour_img = img.copy()
cv2.drawContours(contour_img, [contour], -1, (255, 0, 0), 3)
Image.fromarray(contour_img)

NameError: name 'edge_img' is not defined

In [5]:
def four_point_transform(image, pts):
    def order_points(pts):
        rect = np.zeros((4, 2), dtype = "float32")

        s = pts.sum(axis = 1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]

        diff = np.diff(pts, axis = 1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]

        return rect

    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    return warped

# 对原始图像应用四点透视变换，以获得纸张的俯视图
paper = four_point_transform(img, contour.reshape(4, 2))
Image.fromarray(paper)

NameError: name 'img' is not defined